In [1]:
import os
import fnmatch
import json

In [2]:
# Create destination directory
# source_directory = r'/data/DoseBased_alignment_data_restructured'
# destination_directory = r'/data/DoseBasedAlign_data'
# logfile_directory = r'/home/shahpouriz/Data/output/PXAlignment/log_UID_test.txt'

directory = '/home/shahpouriz/Data/DBP_newDATA/DBP/results_DBP_OP007_Photons_20231024_053647.json'

In [12]:
import json
import glob
import os
class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        self.file_identifier = self.data['patients_file_identifier']
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            print(plan['completed'])
            plan_name = plan['name']
            planning_exam_name = plan['planning_examination']['name']
            dose_dicom_filename = plan['planning_examination']['dose_DICOM_filename']

            evaluation_exams = []
            for exam in plan['evaluation_examinations']:
                for opt_case in exam['optimization_cases']:
                    if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                        final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                        x = round(final_trans_coord.get('x', 0), 5)
                        y = round(final_trans_coord.get('y', 0), 5)
                        z = round(final_trans_coord.get('z', 0), 5)
                        exam_detail = {
                            # 'name': exam['name'],
                            'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                            'dose_dicom_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                        }
                        evaluation_exams.append(exam_detail)

            plan_details.append({
                'plan_name': plan_name,
                'planning_exam_name': planning_exam_name,
                'dose_dicom_filename': dose_dicom_filename,
                'evaluation_examinations': evaluation_exams
            })

        return plan_details

    def get_plan_details(self):
        return self.plan_details
# Usage
file_path = '/home/shahpouriz/Data/DBP_newDATA/DBP/results_DBP_OP007_Photons_20231024_053647.json'
treatment_data = RawDataReader (file_path)
print(treatment_data.id)
print(treatment_data.file_identifier)
print(treatment_data.get_plan_details())


True
DBP_OP007
DBP_OP007_Photons_20231006_182434
[{'plan_name': 'A1FHH', 'planning_exam_name': 'pCTf0', 'dose_dicom_filename': 'DBP_OP007/A1FHH/rtdose_pCTf0_1.2.752.243.1.1.20231023170004754.3603.98635.dcm', 'evaluation_examinations': [{'final_translation_coordinate': {'x': -0.00983, 'y': -0.01466, 'z': -0.05631}, 'dose_dicom_filename': 'DBP_OP007/A1FHH/rCTp2/optimization_case_0/rtdose_rCTp2_1.2.752.243.1.1.20231023170058500.6267.64263.dcm'}, {'final_translation_coordinate': {'x': -0.31608, 'y': 0.57203, 'z': -1.43678}, 'dose_dicom_filename': 'DBP_OP007/A1FHH/rCTp2/optimization_case_1/rtdose_rCTp2_1.2.752.243.1.1.20231023171030719.5034.60642.dcm'}, {'final_translation_coordinate': {'x': -1.24023, 'y': -1.13003, 'z': -0.5508}, 'dose_dicom_filename': 'DBP_OP007/A1FHH/rCTp2/optimization_case_2/rtdose_rCTp2_1.2.752.243.1.1.20231023173055319.6661.43643.dcm'}, {'final_translation_coordinate': {'x': -0.8456, 'y': 1.02861, 'z': 0.76292}, 'dose_dicom_filename': 'DBP_OP007/A1FHH/rCTp2/optimizati

--------------------